In [1]:
import numpy as np
import pandas as pd
import env

import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from sklearn.model_selection import train_test_split
from scipy.stats import spearmanr
from wrangle import get_zillow,prep_zillow,split_data,X_y_split
from explore import plot_categorical_and_continuous,plot_variable_pairs,sub_plot_categorical_and_continuous
warnings.filterwarnings('ignore')
import warnings
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, QuantileTransformer
from pydataset import data
from sklearn.feature_selection import SelectKBest, RFE, f_regression, SequentialFeatureSelector 
from sklearn.linear_model import LinearRegression


In [2]:
tips =sns.load_dataset("tips")

In [3]:
tips["price_per_person"]=round((tips.total_bill/tips["size"]),2)

In [4]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size,price_per_person
0,16.99,1.01,Female,No,Sun,Dinner,2,8.49
1,10.34,1.66,Male,No,Sun,Dinner,3,3.45
2,21.01,3.50,Male,No,Sun,Dinner,3,7.00
3,23.68,3.31,Male,No,Sun,Dinner,2,11.84
4,24.59,3.61,Female,No,Sun,Dinner,4,6.15


In [ ]:
#tips.corr()

In [5]:
new_tips = pd.get_dummies(tips, columns=['time', 'sex', 'day', 'smoker', 'size'])

# Convert specific columns to integer, excluding 'total_bill'
columns_to_convert = ['time_Lunch', 'time_Dinner', 'sex_Female', 'sex_Male', 'day_Thur', 'day_Fri', 'day_Sat', 'day_Sun', 'smoker_Yes', 'smoker_No', 'size_1','size_2','size_3','size_4','size_5','size_6']
new_tips[columns_to_convert] = new_tips[columns_to_convert].astype(int) #Do this for project


In [6]:
new_tips.head()

,total_bill,tip,price_per_person,time_Lunch,time_Dinner,sex_Male,sex_Female,day_Thur,day_Fri,day_Sat,day_Sun,smoker_Yes,smoker_No,size_1,size_2,size_3,size_4,size_5,size_6
0,16.99,1.01,8.49,0,1,0,1,0,0,0,1,0,1,0,1,0,0,0,0
1,10.34,1.66,3.45,0,1,1,0,0,0,0,1,0,1,0,0,1,0,0,0
2,21.01,3.50,7.00,0,1,1,0,0,0,0,1,0,1,0,0,1,0,0,0
3,23.68,3.31,11.84,0,1,1,0,0,0,0,1,0,1,0,1,0,0,0,0
4,24.59,3.61,6.15,0,1,0,1,0,0,0,1,0,1,0,0,0,1,0,0


In [ ]:
tips.info()

In [ ]:
mms_dummy = MinMaxScaler()

In [ ]:
mms_dummy.fit(tips[['total_bill', 'price_per_person']])

In [ ]:
tips[['total_bill', 'price_per_person']] = mms_dummy.transform(tips[['total_bill', 'price_per_person']])

In [ ]:
tips.head()

In [ ]:
train,validate,test = split_data(tips)

In [ ]:
train

In [ ]:
X_train.head()

In [ ]:
# tips.corr()

In [ ]:
# sns.pairplot(data=tips, corner=True, kind="reg", plot_kws={'line_kws':{'color':'red'}})
# plt.show()

In [ ]:
# sns.heatmap(tips.corr(), cmap='PRGn', annot=True,
#            mask=np.triu(tips.corr()))
# plt.show()

In [ ]:
scaler = MinMaxScaler()

In [ ]:
scaler.fit(X_train)

In [ ]:
#use it
#make a new column within train
#use 'transform' instead of 'predict'
X_train_scaled = scaler.transform(X_train)
X_validate_scaled = scaler.transform(X_validate)
X_test_scaled = scaler.transform(X_test)

In [ ]:
X_train

In [ ]:
best_tip = SelectKBest(f_regression, k=2)

In [ ]:
# # Define a mapping dictionary
# time_mapping = {'Lunch': 0, 'Dinner': 1}
# sex_mapping = {'Male': 0, 'Female': 1}
# day_mapping = {'Thur': 0, 'Fri': 1, 'Sat': 2, 'Sun':3}
# smoker_mapping = {'No': 0, 'Yes': 1}
# # Replace 'time' column with mapped integers
# tips['time'] = tips['time'].map(time_mapping)
# tips['sex'] = tips['sex'].map(sex_mapping)
# tips['day'] = tips['day'].map(day_mapping)
# tips['smoker'] = tips['smoker'].map(smoker_mapping)

In [ ]:
#X_train = tips.drop(columns="tip")

In [ ]:
#y_train = tips.tip

In [ ]:
best_tip.fit(X_train,y_train)

In [ ]:
best_tip.scores_

In [ ]:
best_tip.get_feature_names_out()

In [ ]:
ruffy = RFE(LinearRegression(), n_features_to_select=2)

In [ ]:
ruffy.fit(X_train,y_train)

In [ ]:
ruffy.ranking_

In [ ]:
new =pd.DataFrame(
{
    'feature': X_train.columns.to_list(),
    'rfe_ranking': ruffy.ranking_
    
})

In [ ]:
new

In [ ]:
ruffyB = RFE(LinearRegression(), n_features_to_select=3)

In [ ]:
ruffyB.fit(X_train,y_train)

In [ ]:
ruffyB.ranking_

In [ ]:
newB =pd.DataFrame(
{
    'feature': X_train.columns.to_list(),
    'rfe_ranking': ruffyB.ranking_
    
})

In [ ]:
newB

In [ ]:
def select_kbest(X, y, k):
    best_tip = SelectKBest(f_regression, k=k)
    best_tip.fit(X, y)
    selected_feature_names = X.columns[best_tip.get_support(indices=True)].tolist()

    return f"The best {k} features are: {selected_feature_names}"



In [ ]:
select_kbest(X_train,y_train,2)

In [ ]:


def besty_rfe(X, y, k):
    besty_rfe = RFE(LinearRegression(), n_features_to_select=k)
    besty_rfe.fit(X, y)
    selected_feature_names = X.columns[besty_rfe.get_support(indices=True)].tolist()

    print(f"The best {k} features are: {selected_feature_names}")

    ranking = pd.DataFrame(
        {
            'feature': X.columns.to_list(),
            'rfe_ranking': besty_rfe.ranking_
        })

    return ranking


In [ ]:
besty_rfe(X_train,y_train,2)

In [ ]:
swiss = data("swiss")

In [ ]:
swiss.head()

In [ ]:
train2,test2,split2 = split_data(swiss)

In [ ]:
train2.head()

In [ ]:
X_train2 =train2.drop(columns="Fertility")

In [ ]:
y_train2 = train2.Fertility

In [ ]:
select_kbest(X_train2,y_train2,3)

In [ ]:
besty_rfe(X_train2,y_train2,3)